In [1]:
import tensorflow as tf
import yaml
import os
from object_detection.utils import dataset_util


#flags = tf.app.flags
#flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
#FLAGS = flags.FLAGS

LABEL_DICT =  {
    "Green" : 1,
    "Red" : 2,
    "Yellow" : 3,
    "off" : 4,
    }


In [2]:
def create_tf_example(example):
    # TODO(user): Populate the following variables from your example.
    height = 600 # Image height
    width = 800 # Image width
    filename = example['filename'] # Filename of the image. Empty if image is not from file
    filename = filename.encode()
    encoded_image_data = None # Encoded image bytes

    with tf.gfile.GFile(example['filename'], 'rb') as fd:
        encoded_image_data = fd.read()

    image_format = b'jpeg' # b'jpeg' or b'png'

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)

    for box in example['annotations']:
        xmins.append(float(box['xmin'] / width))
        xmaxs.append(float((box['xmin'] + box['x_width']) / width))
        ymins.append(float(box['ymin'] / height))
        ymaxs.append(float((box['ymin']+ box['y_height']) / height))
        classes_text.append(box['class'].encode())
        classes.append(int(LABEL_DICT[box['class']]))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [7]:
def main(_):
    
    #writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    writer = tf.python_io.TFRecordWriter("sim_data.record")

    # TODO(user): Write code to read in your dataset to examples variable
    annotations_file = "../../CarND-Capstone/data/sim_training_data/sim_data_annotations.yaml"
    examples = yaml.load(open(annotations_file, 'rb').read())
    
    #print("type(examples) ", type(examples))
    
    for i in range(len(examples)):
        examples[i]['filename'] = os.path.dirname(annotations_file) + "/" + examples[i]['filename']

    #print("examples 0:2", examples[0:2])
    for example in examples:
        #print(example)
        tf_example = create_tf_example(example)
        writer.write(tf_example.SerializeToString())

    writer.close()
    print("TF record generation complete")

In [8]:
if __name__ == '__main__':
  tf.app.run()

TF record generation complete


SystemExit: 

/Users/chisolm/anaconda/envs/carnd-term1/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
%tb

SystemExit: 